In [1]:
import numpy as np
from openfermion.transforms import get_sparse_operator
from openfermion.utils import pauli_exp_to_qasm, uccsd_singlet_paramsize, uccsd_singlet_generator
from openfermion.ops import QubitOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from pyscf import ci
from pyscf import mp
from openfermionpyscf import run_pyscf
from openfermion.hamiltonians import MolecularData
import os

from numpy import array, concatenate, zeros
from numpy.random import randn
from scipy.optimize import minimize

from openfermion.config import *

from openfermion.hamiltonians import MolecularData
from openfermion.transforms import jordan_wigner
from openfermion.utils import uccsd_singlet_paramsize

In [8]:
geometry = [["H", [0, 0, 0]],
            ["H", [0, 0, 0.7414]]]
basis = "6-31g"
multiplicity = 1
charge = 0
molecule = MolecularData(geometry, basis, multiplicity, charge)
h2_molecule = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)

In [9]:
kubit=h2_molecule.n_qubits
orbital=h2_molecule.n_orbitals
print(kubit)
print(orbital)

8
4


In [4]:
elektron=h2_molecule.n_electrons
print(elektron)

2


In [10]:
b=h2_molecule.get_molecular_hamiltonian(occupied_indices=[0,2], active_indices=[0,1,2,3])
print(b)

() 0.8877379136285546
((0, 1), (0, 0)) 0.35895916506914105
((0, 1), (4, 0)) 0.11985114991659893
((1, 1), (1, 0)) 0.35895916506914105
((1, 1), (5, 0)) 0.11985114991659893
((2, 1), (2, 0)) 0.9649437622838949
((2, 1), (6, 0)) -0.19412157979774922
((3, 1), (3, 0)) 0.9649437622838949
((3, 1), (7, 0)) -0.19412157979774922
((4, 1), (0, 0)) 0.11985114991659908
((4, 1), (4, 0)) 1.239073115475216
((5, 1), (1, 0)) 0.11985114991659908
((5, 1), (5, 0)) 1.239073115475216
((6, 1), (2, 0)) -0.19412157979774886
((6, 1), (6, 0)) 2.3794114767767884
((7, 1), (3, 0)) -0.19412157979774886
((7, 1), (7, 0)) 2.3794114767767884
((0, 1), (0, 1), (0, 0), (0, 0)) 0.3248513173560207
((0, 1), (0, 1), (0, 0), (4, 0)) 0.08353666530450171
((0, 1), (0, 1), (2, 0), (2, 0)) 0.04007327150167787
((0, 1), (0, 1), (2, 0), (6, 0)) -0.03968822445145349
((0, 1), (0, 1), (4, 0), (0, 0)) 0.08353666530450171
((0, 1), (0, 1), (4, 0), (4, 0)) 0.05465044919893638
((0, 1), (0, 1), (6, 0), (2, 0)) -0.03968822445145348
((0, 1), (0, 1), (

In [17]:
from openfermion.ops import FermionOperator
x=uccsd_singlet_paramsize(h2_molecule.n_qubits,h2_molecule.n_electrons)
print(x)
z_1=FermionOperator('0^ 1^')
print(bravyi_kitaev(z_1))

9
(0.25+0j) [X0] +
(0.25+0j) [X0 Z1] +
-0.25j [Y0] +
-0.25j [Y0 Z1]


In [6]:
import numpy as np
a=np.linspace(1,2,10)
t=uccsd_singlet_generator(a,h2_molecule.n_qubits,h2_molecule.n_electrons)

In [7]:
print(t)

-1.0 [0^ 2] +
-1.1111111111111112 [0^ 2 1^ 3] +
-1.0 [1^ 3] +
-1.1111111111111112 [1^ 3 0^ 2] +
1.0 [2^ 0] +
1.1111111111111112 [2^ 0 3^ 1] +
1.0 [3^ 1] +
1.1111111111111112 [3^ 1 2^ 0]


In [8]:
import openfermionpyscf as ofpyscf
import openfermion as of
import cirq
import openfermioncirq as ofc
import numpy as np
import pyscf

In [9]:
from openfermion.hamiltonians import MolecularData
molecule = MolecularData(geometry, basis, multiplicity, charge)
h2_molecule = ofpyscf.run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)
elektron=h2_molecule.n_electrons
orbitals = molecule.n_orbitals
qubit= molecule.n_qubits

In [10]:
initial_amplitudes = np.linspace(1,2,2)
#membuat ansatz UCCSD
uccsd_fermion=uccsd_singlet_generator(initial_amplitudes,qubit,elektron)
print(uccsd_fermion)

-1.0 [0^ 2] +
-2.0 [0^ 2 1^ 3] +
-1.0 [1^ 3] +
-2.0 [1^ 3 0^ 2] +
1.0 [2^ 0] +
2.0 [2^ 0 3^ 1] +
1.0 [3^ 1] +
2.0 [3^ 1 2^ 0]


In [11]:
from pyquil.paulis import exponentiate

In [12]:
from pyquil.quil import Program
import pyquil.api as api
from pyquil.gates import *
from pyquil.api import get_qc

In [13]:
prog=Program()

In [14]:
from openfermion.ops import FermionOperator

In [15]:
a_1=FermionOperator('2^ 0')
a_2=FermionOperator('0^ 2')
a_3=FermionOperator('2^ 1')
a_4=FermionOperator('1^ 2')
a_5=FermionOperator('3^ 0')
a_6=FermionOperator('0^ 3')
a_7=FermionOperator('3^ 1')
a_8=FermionOperator('1^ 3')
a_9=FermionOperator('4^ 0')
a_10=FermionOperator('0^ 4')
a_11=FermionOperator('4^ 1')
a_12=FermionOperator('1^ 4')
a_13=FermionOperator('5^ 0')
a_14=FermionOperator('0^ 5')
a_15=FermionOperator('5^ 1')
a_16=FermionOperator(' 1^ 5')



d_1=FermionOperator('2^ 3^ 0 1')
d_2=FermionOperator('1^ 0^ 3 2')
d_3=FermionOperator('3^ 2^ 1 0')
d_4=FermionOperator('0^ 1^ 2 3')

d_5=FermionOperator('4^ 5^ 0 1')
d_6=FermionOperator('1^ 0^ 5 4')
d_7=FermionOperator('5^ 4^ 1 0')
d_8=FermionOperator('0^ 1^ 4 5')


In [16]:
s1=1*(a_1-a_2)
s2=1*(a_3-a_4)
s3=1*(a_5-a_6)
s4=1*(a_7-a_8)
s5=1*(d_1-d_2)
s6=1*(d_3-d_4)
to=s6
print(to)

-1.0 [0^ 1^ 2 3] +
1.0 [3^ 2^ 1 0]


In [17]:
s7=1*(a_9-a_10)
s8=1*(a_11-a_12)
s9=1*(a_13-a_14)
s10=1*(a_15-a_16)
s11=1*(d_5-d_6)
s12=1*(d_7-d_8)
to=s12
print(to)

-1.0 [0^ 1^ 4 5] +
1.0 [5^ 4^ 1 0]


In [18]:
suku1=jordan_wigner(s1)
print(suku1)

-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2]


In [19]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X0 Z1 Y2 ' , -0.5 ) ,QubitOperator ( ' Y0 Z1 X2 ' , 0.5 ) ] ) :
    print(line)

H 0
Rx 1.5707963267948966 2
CNOT 0 1
CNOT 1 2
Rz -0.5 2
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 2
Rx 1.5707963267948966 0
H 2
CNOT 0 1
CNOT 1 2
Rz 0.5 2
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 2


In [20]:
suku2=jordan_wigner(s2)
print(suku2)


-0.5j [X1 Y2] +
0.5j [Y1 X2]


In [21]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X1 Y2 ' , -0.5 ) ,QubitOperator ( ' Y1 X2 ' , 0.5 ) ] ) :
    print(line)

H 1
Rx 1.5707963267948966 2
CNOT 1 2
Rz -0.5 2
CNOT 1 2
H 1
Rx -1.5707963267948966 2
Rx 1.5707963267948966 1
H 2
CNOT 1 2
Rz 0.5 2
CNOT 1 2
Rx -1.5707963267948966 1
H 2


In [22]:
suku3=jordan_wigner(s3)
print(suku3)

-0.5j [X0 Z1 Z2 Y3] +
0.5j [Y0 Z1 Z2 X3]


In [23]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X0 Z1 Z2 Y3 ' , -0.5 ) ,QubitOperator ( ' Y0 Z1 Z2 X2 ' , 0.5 ) ] ) :
    print(line)

H 0
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 3
Rx 1.5707963267948966 0
Rx 1.5707963267948966 2
CNOT 0 1
CNOT 1 2
Rz 0.0 2
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
Rx -1.5707963267948966 2


In [24]:
suku4=jordan_wigner(s4)
print(suku4)

-0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]


In [25]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X1 Z2 Y3 ' , -0.5 ) ,QubitOperator ( ' Y1 Z2 X3 ' , 0.5 ) ] ) :
    print(line)

H 1
Rx 1.5707963267948966 3
CNOT 1 2
CNOT 2 3
Rz -0.5 3
CNOT 2 3
CNOT 1 2
H 1
Rx -1.5707963267948966 3
Rx 1.5707963267948966 1
H 3
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
Rx -1.5707963267948966 1
H 3


In [26]:
suku5=jordan_wigner(s5)
print(suku5)

0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]


In [27]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X0 X1 X2 Y3 ' , -0.5 ) ,QubitOperator ( ' X0 X1 Y2 X3 ' , 0.5 ), 
                                QubitOperator('X0 Y1 X2 X3',0.5), QubitOperator('X0 Y1 Y2 Y3',0.5), QubitOperator
                                ('Y0 X1 X2 X3',0.5),QubitOperator('Y0 X1 Y2 Y3',0.5),QubitOperator('Y0 Y1 X2 Y3',0.5),
                                QubitOperator('Y0 Y1 Y2 X3',0.5)] ) :
    print(line)

H 0
H 1
H 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
H 2
Rx -1.5707963267948966 3
H 0
H 1
Rx 1.5707963267948966 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
Rx -1.5707963267948966 2
H 3
H 0
Rx 1.5707963267948966 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 1
H 2
H 3
H 0
Rx 1.5707963267948966 1
Rx 1.5707963267948966 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 1
Rx -1.5707963267948966 2
Rx -1.5707963267948966 3
Rx 1.5707963267948966 0
H 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 1
H 2
H 3
Rx 1.5707963267948966 0
H 1
Rx 1.5707963267948966 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 1
Rx -1.5707963267948966 2
Rx -1.5707963267948966 3
Rx 1.57079632679489

In [28]:
suku6=jordan_wigner(s6)
print(suku6)

0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]


In [29]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X0 X1 X2 Y3 ' , -0.5 ) ,QubitOperator ( ' X0 X1 Y2 X3 ' , 0.5 ), 
                                QubitOperator('X0 Y1 X2 X3',0.5), QubitOperator('X0 Y1 Y2 Y3',0.5), QubitOperator
                                ('Y0 X1 X2 X3',0.5),QubitOperator('Y0 X1 Y2 Y3',0.5),QubitOperator('Y0 Y1 X2 Y3',0.5),
                                QubitOperator('Y0 Y1 Y2 X3',0.5)] ) :
    print(line)

H 0
H 1
H 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
H 2
Rx -1.5707963267948966 3
H 0
H 1
Rx 1.5707963267948966 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
Rx -1.5707963267948966 2
H 3
H 0
Rx 1.5707963267948966 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 1
H 2
H 3
H 0
Rx 1.5707963267948966 1
Rx 1.5707963267948966 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 1
Rx -1.5707963267948966 2
Rx -1.5707963267948966 3
Rx 1.5707963267948966 0
H 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 1
H 2
H 3
Rx 1.5707963267948966 0
H 1
Rx 1.5707963267948966 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.5 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 1
Rx -1.5707963267948966 2
Rx -1.5707963267948966 3
Rx 1.57079632679489

In [30]:
suku7=jordan_wigner(s7)
print(suku7)

-0.5j [X0 Z1 Z2 Z3 Y4] +
0.5j [Y0 Z1 Z2 Z3 X4]


In [32]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X0 Z1 Z2 Z3 Y4' , -0.5 ) ,QubitOperator ( 'Y0 Z1 Z2 Z3 X4 ' , 0.5 ), 
                ] ) :
    print(line)

H 0
Rx 1.5707963267948966 4
CNOT 0 1
CNOT 1 2
CNOT 2 3
CNOT 3 4
Rz -0.5 4
CNOT 3 4
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 4
Rx 1.5707963267948966 0
H 4
CNOT 0 1
CNOT 1 2
CNOT 2 3
CNOT 3 4
Rz 0.5 4
CNOT 3 4
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 4


In [34]:
suku8=jordan_wigner(s8)
print(suku8)

-0.5j [X1 Z2 Z3 Y4] +
0.5j [Y1 Z2 Z3 X4]


In [35]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X1 Z2 Z3 Y4' , -0.5 ) ,QubitOperator ( 'Y1 Z2 Z3 X4 ' , 0.5 ), 
                ] ) :
    print(line)

H 1
Rx 1.5707963267948966 4
CNOT 1 2
CNOT 2 3
CNOT 3 4
Rz -0.5 4
CNOT 3 4
CNOT 2 3
CNOT 1 2
H 1
Rx -1.5707963267948966 4
Rx 1.5707963267948966 1
H 4
CNOT 1 2
CNOT 2 3
CNOT 3 4
Rz 0.5 4
CNOT 3 4
CNOT 2 3
CNOT 1 2
Rx -1.5707963267948966 1
H 4


In [36]:
suku9=jordan_wigner(s9)
print(suku9)

-0.5j [X0 Z1 Z2 Z3 Z4 Y5] +
0.5j [Y0 Z1 Z2 Z3 Z4 X5]


In [37]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X0 Z1 Z2 Z3 Z4 Y5' , -0.5 ) ,QubitOperator ( 'Y0 Z1 Z2 Z3 Z4 X5 ' , 0.5 ), 
                ] ) :
    print(line)

H 0
Rx 1.5707963267948966 5
CNOT 0 1
CNOT 1 2
CNOT 2 3
CNOT 3 4
CNOT 4 5
Rz -0.5 5
CNOT 4 5
CNOT 3 4
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 5
Rx 1.5707963267948966 0
H 5
CNOT 0 1
CNOT 1 2
CNOT 2 3
CNOT 3 4
CNOT 4 5
Rz 0.5 5
CNOT 4 5
CNOT 3 4
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 5


In [38]:
suku10=jordan_wigner(s10)
print(suku10)

-0.5j [X1 Z2 Z3 Z4 Y5] +
0.5j [Y1 Z2 Z3 Z4 X5]


In [39]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X1 Z2 Z3 Z4 Y5' , -0.5 ) ,QubitOperator ( 'Y1 Z2 Z3 Z4 X5 ' , 0.5 ), 
                ] ) :
    print(line)

H 1
Rx 1.5707963267948966 5
CNOT 1 2
CNOT 2 3
CNOT 3 4
CNOT 4 5
Rz -0.5 5
CNOT 4 5
CNOT 3 4
CNOT 2 3
CNOT 1 2
H 1
Rx -1.5707963267948966 5
Rx 1.5707963267948966 1
H 5
CNOT 1 2
CNOT 2 3
CNOT 3 4
CNOT 4 5
Rz 0.5 5
CNOT 4 5
CNOT 3 4
CNOT 2 3
CNOT 1 2
Rx -1.5707963267948966 1
H 5


In [41]:
suku11=jordan_wigner(s11)
print(suku11)

0.125j [X0 X1 X4 Y5] +
0.125j [X0 X1 Y4 X5] +
-0.125j [X0 Y1 X4 X5] +
0.125j [X0 Y1 Y4 Y5] +
-0.125j [Y0 X1 X4 X5] +
0.125j [Y0 X1 Y4 Y5] +
-0.125j [Y0 Y1 X4 Y5] +
-0.125j [Y0 Y1 Y4 X5]


In [42]:
for line in pauli_exp_to_qasm ( [ QubitOperator ( 'X0 X1 X4 Y5 ' , -0.5 ) ,QubitOperator ( ' X0 X1 Y4 X5' , 0.5 ), 
                                QubitOperator('X0 Y1 X4 X5',0.5), QubitOperator('X0 Y1 Y4 Y5',0.5), QubitOperator
                                ('Y0 X1 X4 X5',0.5),QubitOperator('Y0 X1 Y4 Y5',0.5),QubitOperator('Y0 Y1 X4 Y5',0.5),
                                QubitOperator('Y0 Y1 Y4 X5',0.5)] ) :
    print(line)

H 0
H 1
H 4
Rx 1.5707963267948966 5
CNOT 0 1
CNOT 1 4
CNOT 4 5
Rz -0.5 5
CNOT 4 5
CNOT 1 4
CNOT 0 1
H 0
H 1
H 4
Rx -1.5707963267948966 5
H 0
H 1
Rx 1.5707963267948966 4
H 5
CNOT 0 1
CNOT 1 4
CNOT 4 5
Rz 0.5 5
CNOT 4 5
CNOT 1 4
CNOT 0 1
H 0
H 1
Rx -1.5707963267948966 4
H 5
H 0
Rx 1.5707963267948966 1
H 4
H 5
CNOT 0 1
CNOT 1 4
CNOT 4 5
Rz 0.5 5
CNOT 4 5
CNOT 1 4
CNOT 0 1
H 0
Rx -1.5707963267948966 1
H 4
H 5
H 0
Rx 1.5707963267948966 1
Rx 1.5707963267948966 4
Rx 1.5707963267948966 5
CNOT 0 1
CNOT 1 4
CNOT 4 5
Rz 0.5 5
CNOT 4 5
CNOT 1 4
CNOT 0 1
H 0
Rx -1.5707963267948966 1
Rx -1.5707963267948966 4
Rx -1.5707963267948966 5
Rx 1.5707963267948966 0
H 1
H 4
H 5
CNOT 0 1
CNOT 1 4
CNOT 4 5
Rz 0.5 5
CNOT 4 5
CNOT 1 4
CNOT 0 1
Rx -1.5707963267948966 0
H 1
H 4
H 5
Rx 1.5707963267948966 0
H 1
Rx 1.5707963267948966 4
Rx 1.5707963267948966 5
CNOT 0 1
CNOT 1 4
CNOT 4 5
Rz 0.5 5
CNOT 4 5
CNOT 1 4
CNOT 0 1
Rx -1.5707963267948966 0
H 1
Rx -1.5707963267948966 4
Rx -1.5707963267948966 5
Rx 1.57079632679489

In [43]:
suku12=jordan_wigner(s12)
print(suku12)

0.125j [X0 X1 X4 Y5] +
0.125j [X0 X1 Y4 X5] +
-0.125j [X0 Y1 X4 X5] +
0.125j [X0 Y1 Y4 Y5] +
-0.125j [Y0 X1 X4 X5] +
0.125j [Y0 X1 Y4 Y5] +
-0.125j [Y0 Y1 X4 Y5] +
-0.125j [Y0 Y1 Y4 X5]


In [ ]:
0.125j [X0 X1 X4 Y5] +
0.125j [X0 X1 Y4 X5] +
-0.125j [X0 Y1 X4 X5] +
0.125j [X0 Y1 Y4 Y5] +
-0.125j [Y0 X1 X4 X5] +
0.125j [Y0 X1 Y4 Y5] +
-0.125j [Y0 Y1 X4 Y5] +
-0.125j [Y0 Y1 Y4 X5]